# Custom Supervisor Example

I want to make a version of an agent supervisor system that i have made, implementing the supervisor from scratch. 

This combines elements for the agent_supervisor, InjectedState and Reducers notebooks. See those for references.

We will implement an extremely simplified version of our RAG system, that will just load datasets into state, analize them and visualize them. In order to do so we will have just a supervisor and a data analyst with visualization capabilities.